In [1]:
import pandas as pd

In [2]:
# patient list

patient_df = pd.read_excel('/Users/yena/Desktop/SCH/척척석사/EMR related/낙상데이터/천안/(천안병원)간호간병통합병동 낙상대상자 리스트(2017-2021.10).xlsx', index_col =0)
patient_list = patient_df['등록번호'].astype(str).unique()  # 등록번호 unique -> 228명 확인
fall_date = patient_df[['등록번호', '낙상발생일시']]

duration=[]

for i in fall_date['낙상발생일시'].tolist():
    i = str(i)
    i = i.split(' ')[0]
    duration.append(i)

fall_date['낙상발생일시'] = duration


/var/folders/bj/hzh5z8r11fz8jg3x13jvphq00000gn/T/ipykernel_8206/1648803713.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fall_date['낙상발생일시'] = duration


In [3]:
patient_cci = pd.read_csv('/Users/yena/Desktop/정리/github/천안_cci.csv', index_col=0)
basic_patient = pd.read_csv('/Users/yena/Desktop/정리/github/천안_기초정보.csv',index_col=0)
pain_df = pd.read_csv('/Users/yena/Desktop/정리/github/천안_통증.csv', index_col=0)
act_nut_patient = pd.read_csv('/Users/yena/Desktop/정리/github/천안_욕창.csv',index_col=0)
final_adl = pd.read_csv('/Users/yena/Desktop/정리/github/천안_간호필요도.csv',index_col=0)
final_fall = pd.read_csv('/Users/yena/Desktop/정리/github/천안_낙상위험도평가_MFS.csv', index_col=0)
atc_drug = pd.read_csv('/Users/yena/Desktop/정리/github/천안_낙상_약물.csv', index_col=0)

In [4]:
basic_patient = basic_patient.rename(columns={'환자번호':'등록번호'})
patient_cci = patient_cci.rename(columns={'환자번호':'등록번호'})

In [5]:
print('환자 수', len(patient_list))
print('환자_CCI',len(patient_cci['등록번호'].unique()))
print('환자_BASIC',len(basic_patient['등록번호'].unique()))
print('통증',len(pain_df['등록번호'].unique()))
print('욕창', len(act_nut_patient['등록번호'].unique()))
print('간호필요도', len(final_adl['등록번호'].unique()))
print('낙상위험도', len(final_fall['등록번호'].unique()))
print('약물', len(atc_drug['등록번호'].unique()))

환자 수 80
환자_CCI 19613
환자_BASIC 23934
통증 21549
욕창 47
간호필요도 60
낙상위험도 56
약물 77


In [6]:
patient_cci = patient_cci.astype(str)
basic_patient = basic_patient.astype(str)
pain_df = pain_df.astype(str)
act_nut_patient = act_nut_patient.astype(str)
final_adl = final_adl.astype(str)
final_fall = final_fall.astype(str)
atc_drug = atc_drug.astype(str)

In [7]:
print('환자 수', len(patient_list))
print('환자_CCI',len(patient_cci['등록번호'].unique()))
print('환자_BASIC',len(basic_patient['등록번호'].unique()))
print('통증',len(pain_df['등록번호'].unique()))
print('욕창', len(act_nut_patient['등록번호'].unique()))
print('간호필요도', len(final_adl['등록번호'].unique()))
print('낙상위험도', len(final_fall['등록번호'].unique()))
print('약물', len(atc_drug['등록번호'].unique()))

환자 수 80
환자_CCI 19613
환자_BASIC 23934
통증 21549
욕창 47
간호필요도 60
낙상위험도 56
약물 77


### 낙상위험도 없는 경우 0

In [8]:
fall_date = fall_date.rename(columns = {'낙상일자':'낙상발생일시'})
fall_date = fall_date.astype(str)

In [9]:
no_MFS = []

for i in fall_date['등록번호'].unique():
    if i not in final_fall['등록번호'].unique():
        no_MFS.append(i)

no_MFS_df = fall_date[fall_date['등록번호'].isin(set(no_MFS))]
no_MFS_df['MFS'] = 0

/var/folders/bj/hzh5z8r11fz8jg3x13jvphq00000gn/T/ipykernel_8206/2021797625.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_MFS_df['MFS'] = 0


In [10]:
no_MFS_df = no_MFS_df.astype(str)

In [11]:
final_fall_v2 = final_fall.merge(no_MFS_df, on=['등록번호', 'MFS', '낙상발생일시'],how='outer'\
).astype(str).replace('NaT','0').replace('nan','0')
print(len(final_fall_v2['등록번호'].unique()))
final_fall_v2.head()

80


,등록번호,낙상발생일시,평가일자,낙상이력,보행보조,이차진료,헤파린록,의식정신상태,걸음/의식,days,MFS
0,432783,2018-06-13,2018-06-10,0,0,1,1,0,0,3 days,35
1,432783,2018-06-13,2018-06-13,0,0,1,1,1,0,0 days,50
2,582033,2018-06-27,2018-06-24,0,1,1,1,0,1,3 days,60
3,582033,2018-08-05,2018-08-05,1,1,0,1,1,1,0 days,85
4,582346,2018-09-19,2018-09-16,0,1,1,1,0,1,3 days,60


### 욕창없는경우 0

In [12]:
no_sore = []

for i in fall_date['등록번호'].unique():
    if i not in act_nut_patient['등록번호'].unique():
        no_sore.append(i)

no_sore_df = fall_date[fall_date['등록번호'].isin(set(no_sore))]
no_sore_df['욕창_행동'] = 0
no_sore_df['욕창_영양'] = 0

/var/folders/bj/hzh5z8r11fz8jg3x13jvphq00000gn/T/ipykernel_8206/1825496252.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_sore_df['욕창_행동'] = 0
/var/folders/bj/hzh5z8r11fz8jg3x13jvphq00000gn/T/ipykernel_8206/1825496252.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_sore_df['욕창_영양'] = 0


In [13]:
no_sore_df = no_sore_df.astype(str)

In [14]:
act_nut_patient_v2 = act_nut_patient.merge(no_sore_df.astype(str), on=['등록번호', '낙상발생일시', '욕창_행동', '욕창_영양'],how='outer'\
).astype(str).replace('NaT','0').replace('nan','0')
print(len(act_nut_patient_v2['등록번호'].unique()))
act_nut_patient_v2.head()

80


,등록번호,낙상발생일시,평가일자,욕창_행동,욕창_영양
0,1393198,2017-07-02,2017-07-02,0,1
1,1393198,2017-07-10,2017-07-09,0,1
2,1331989,2017-09-19,2017-09-16,1,0
3,1331989,2017-09-19,2017-09-17,1,0
4,1331989,2017-09-19,2017-09-18,1,0


### 통증없는경우 0

In [15]:
no_pain = []

for i in fall_date['등록번호'].unique():
    if i not in pain_df['등록번호'].astype(str).unique():
        no_pain.append(i)

no_pain_df = fall_date[fall_date['등록번호'].isin(set(no_pain))]
no_pain_df['등록번호'] = no_pain_df['등록번호'].astype(str)
pain_df['등록번호'] = pain_df['등록번호'].astype(str)
no_pain_df['통증강도'] = 0
no_pain_df = no_pain_df[['등록번호', '통증강도']]
no_pain_df = no_pain_df.astype(str)

/var/folders/bj/hzh5z8r11fz8jg3x13jvphq00000gn/T/ipykernel_8206/1197397177.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_pain_df['등록번호'] = no_pain_df['등록번호'].astype(str)
/var/folders/bj/hzh5z8r11fz8jg3x13jvphq00000gn/T/ipykernel_8206/1197397177.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_pain_df['통증강도'] = 0


In [16]:
pain_df_v2 = pain_df.merge(no_pain_df, on=['등록번호', '통증강도'],how='outer').drop_duplicates()

In [17]:
len(pain_df_v2['등록번호'])

21553

#### 기본, 통증, 낙상, 욕창

In [18]:
fall_MFS = pd.DataFrame(final_fall_v2[['등록번호','MFS']].astype(int).groupby(['등록번호'])['MFS'].mean()).reset_index().astype(int)
fall_MFS = fall_MFS.astype(str)

In [19]:
print('환자 수', len(patient_list))
print('환자_CCI',len(patient_cci['등록번호'].unique()))
print('환자_BASIC',len(basic_patient['등록번호'].unique()))
print('통증',len(pain_df_v2['등록번호'].unique()))
print('욕창', len(act_nut_patient_v2['등록번호'].unique()))
print('간호필요도', len(final_adl['등록번호'].unique()))
print('낙상위험도', len(fall_MFS['등록번호'].unique()))
print('약물', len(atc_drug['등록번호'].unique()))

환자 수 80
환자_CCI 19613
환자_BASIC 23934
통증 21553
욕창 80
간호필요도 60
낙상위험도 80
약물 77


#### 약물, MFS, CCI, 간호필요도, 통증, 욕창, 환자 

In [20]:
patients = atc_drug['등록번호'].unique()

In [21]:
basic_patient = basic_patient[basic_patient['등록번호'].isin(set(patients))]
final_adl = final_adl[final_adl['등록번호'].isin(set(patients))]
final_fall = final_fall[final_fall['등록번호'].isin(set(patients))]
act_nut_patient = act_nut_patient_v2[act_nut_patient_v2['등록번호'].isin(set(patients))]
pain = pain_df_v2[pain_df_v2['등록번호'].isin(set(patients))]
atc_drug = atc_drug[atc_drug['등록번호'].isin(set(patients))]
patient_cci = patient_cci[patient_cci['등록번호'].isin(set(patients))]
fall_MFS = fall_MFS[fall_MFS['등록번호'].isin(set(patients))]

In [22]:
print('환자 수', len(patient_list))
print('환자_CCI',len(patient_cci['등록번호'].unique()))
print('환자_BASIC',len(basic_patient['등록번호'].unique()))
print('통증',len(pain_df['등록번호'].unique()))
print('욕창', len(act_nut_patient['등록번호'].unique()))
print('간호필요도', len(final_adl['등록번호'].unique()))
print('낙상위험도', len(fall_MFS['등록번호'].unique()))
print('약물', len(atc_drug['등록번호'].unique()))
print('MFS', len(fall_MFS['등록번호'].unique()))

환자 수 80
환자_CCI 69
환자_BASIC 71
통증 21549
욕창 77
간호필요도 59
낙상위험도 77
약물 77
MFS 77


In [23]:
final_basic = fall_MFS.merge(patient_cci, on=['등록번호'],how='outer').fillna(0).merge(basic_patient, on=['등록번호'], how='inner').drop_duplicates()

In [24]:
final_basic

,등록번호,MFS,total_cci,성별,나이,키,몸무게
0,43940,0,3.8,F,58.25,139.98,67.0
1,108293,0,3.5,M,66.33,140.27,95.5
2,122348,50,0.5,F,81.0,157.0,54.5
3,239823,60,0.0,F,84.0,150.3,53.1
4,330121,0,0,M,63.0,159.2,70.0
...,...,...,...,...,...,...,...
66,1673921,0,7.0,F,21.0,168.9,58.0
67,1686651,47,2.0,M,73.0,157.3,59.0
68,1686972,75,2.5,F,85.0,136.8,40.1
69,3055540,65,3.6,M,61.0,124.62,53.1


In [26]:
fall_MFS.merge(patient_cci, on=['등록번호'],how='outer').fillna(0).merge(basic_patient, on=['등록번호'], how='inner').to_csv('FINAL_천안_기초정보.csv')

In [27]:
print(len(final_basic['등록번호'].unique()))

## 간호필요도
final_basic_adl = final_basic.merge(final_adl, on=['등록번호'], how='outer').drop_duplicates()
print(len(final_basic_adl['등록번호'].unique()))

# ## 약물
final_basic_adl_drug = final_basic_adl.merge(atc_drug, on=['등록번호','평가일자'], how='inner')
print(len(final_basic_adl_drug['등록번호'].unique()))

## 통증
final_basic_adl_drug_pain = final_basic_adl_drug.merge(pain_df, on=['등록번호',], how='inner')
print(len(final_basic_adl_drug_pain['등록번호'].unique()))

## 욕창

final_basic_adl_drug_pain_sore = final_basic_adl_drug_pain.merge(act_nut_patient, on=['등록번호','평가일자','낙상발생일시'], how='inner')
print(len(final_basic_adl_drug_pain_sore['등록번호'].unique()))


71
75
59
56
42


### ADL_추가

In [28]:
#### special_info_df
adl_danger_df = pd.read_csv('천안_위험행동관리.csv')
adl_spe_df = pd.read_csv('천안_전문치료.csv')

final_basic_adl_drug_pain_sore = final_basic_adl_drug_pain_sore.astype(str)
adl_danger_df = adl_danger_df.astype(str)
adl_spe_df = adl_spe_df.astype(str)


In [29]:
adl_danger_df = adl_danger_df.drop(columns=['평가일자'])
adl_spe_df = adl_spe_df.drop(columns=['평가일자'])

In [32]:
adl_danger_df = adl_danger_df.drop_duplicates()
adl_danger_df = adl_danger_df[adl_danger_df['ADL_위험']=='1']
adl_danger_df = adl_danger_df[['등록번호','ADL_위험']]
adl_danger_df

,등록번호,ADL_위험
15,1529576,1
16,1529576,1


In [34]:
adl_spe_df = adl_spe_df.drop_duplicates()
adl_spe_df = adl_spe_df[adl_spe_df['ADL_전문']=='1']
adl_spe_df = adl_spe_df[['등록번호','ADL_전문']]
adl_spe_df

,등록번호,ADL_전문
0,1393198,1
1,1393198,1
2,1393198,1
3,1393198,1
4,1393198,1
5,1393198,1
6,1393198,1
7,1393198,1
8,1393198,1
9,1393198,1


In [35]:
new_adl_danger_p = adl_danger_df['등록번호'].tolist()
new_adl_danger_d = adl_danger_df['ADL_위험'].tolist()

for i in patient_list:
    if i not in adl_danger_df['등록번호'].unique():
        new_adl_danger_p.append(i)
        new_adl_danger_d.append(0)

danger_df = pd.DataFrame({'등록번호':new_adl_danger_p, 'ADL_위험':new_adl_danger_d})
danger_df

,등록번호,ADL_위험
0,1529576,1
1,1529576,1
2,1393198,0
3,1384838,0
4,1474897,0
...,...,...
76,594511,0
77,860045,0
78,360023,0
79,1323071,0


In [36]:
new_adl_spe_p = adl_spe_df['등록번호'].tolist()
new_adl_spe_d = adl_spe_df['ADL_전문'].tolist()

for i in patient_list:
    if i not in adl_spe_df['등록번호'].unique():
        new_adl_spe_p.append(i)
        new_adl_spe_d.append(0)

spe_df = pd.DataFrame({'등록번호':new_adl_spe_p, 'ADL_전문':new_adl_spe_d})
spe_df

,등록번호,ADL_전문
0,1393198,1
1,1393198,1
2,1393198,1
3,1393198,1
4,1393198,1
...,...,...
102,594511,0
103,860045,0
104,360023,0
105,1323071,0


In [37]:
len(final_basic_adl_drug_pain_sore['등록번호'].unique())

42

In [38]:
final_basic_adl_drug_pain_sore.shape

(339, 20)

In [39]:
final_basic_adl_drug_pain_sore_danger = final_basic_adl_drug_pain_sore.merge(danger_df, on=['등록번호'], how='inner')
final_basic_adl_drug_pain_sore_danger_spe = final_basic_adl_drug_pain_sore_danger.merge(spe_df, on=['등록번호'], how='inner')
final_basic_adl_drug_pain_sore_danger_spe

,등록번호,MFS,total_cci,성별,나이,키,몸무게,평가일자,"ADL_배뇨, 배변",ADL_체위변경,...,호흡 간호_비침습적 산소 투여,낙상발생일시,유무,Cluster,Cluster_c,통증강도,욕창_행동,욕창_영양,ADL_위험,ADL_전문
0,108293,0,3.5,M,66.33,140.27,95.5,2020-11-29,0,0,...,0,2020-11-29,0 days,B,1,0.6,1,0,0,0
1,108293,0,3.5,M,66.33,140.27,95.5,2020-11-29,0,0,...,0,2020-11-29,0 days,C,2,0.6,1,0,0,0
2,108293,0,3.5,M,66.33,140.27,95.5,2020-11-29,0,0,...,0,2020-11-29,0 days,E,1,0.6,1,0,0,0
3,108293,0,3.5,M,66.33,140.27,95.5,2020-11-29,0,0,...,0,2020-11-29,0 days,G,1,0.6,1,0,0,0
4,108293,0,3.5,M,66.33,140.27,95.5,2020-11-29,0,0,...,0,2020-11-29,0 days,K,1,0.6,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
626,1200546,nan,nan,nan,nan,nan,nan,2019-08-06,0,0,...,0,2019-08-09,3 days,S,1,3.0,0,0,0,0
627,860045,nan,nan,nan,nan,nan,nan,2021-07-25,1,0,...,0,2021-07-25,0 days,C,2,3.5,1,0,0,0
628,860045,nan,nan,nan,nan,nan,nan,2021-07-25,1,0,...,0,2021-07-25,0 days,E,1,3.5,1,0,0,0
629,860045,nan,nan,nan,nan,nan,nan,2021-07-25,1,0,...,0,2021-07-25,0 days,L,1,3.5,1,0,0,0


In [40]:
final_basic_adl_drug_pain_sore_danger_spe.to_csv('FINAL_천안_v2.csv')